In [1]:
import re
import json
import os

In [2]:
import pandas as pd

In [8]:
import glob

In [3]:
%load_ext autoreload
%autoreload 2

In [106]:
from legal_document_splitter import *

In [6]:
DATA_PATH="data/"

### Load all the legal documents from the folder

In [17]:
all_files = np.sort(glob.glob(DATA_PATH + "normas_referenciadas/*.txt"))

In [18]:
all_files

array(['data/normas_referenciadas/ADI RFB nº 12, de 2016.txt',
       'data/normas_referenciadas/Acordo para Evitar a Dupla Tributação em Matéria de Impostos sobre a Renda e o Capital firmado entre o Brasil e a Alemanha.txt',
       'data/normas_referenciadas/Acórdão do RE nº 855.091RS (Tema 808).txt',
       'data/normas_referenciadas/Ato Declaratório (AD) PGFN nº 3, de 18 de setembro de 2008.txt',
       'data/normas_referenciadas/Ato Declaratório Cosar nº 47.txt',
       'data/normas_referenciadas/Ato Declaratório Executivo Codac nº 23, de 4 de dezembro de 2019.txt',
       'data/normas_referenciadas/Ato Declaratório Executivo RFB nº 2, de 6 de março de 2024.txt',
       'data/normas_referenciadas/Ato Declaratório Executivo SRF nº 72, de 22 de dezembro de 2005.txt',
       'data/normas_referenciadas/Ato Declaratório Interpretativo RFB nº 1.txt',
       'data/normas_referenciadas/Ato Declaratório Interpretativo SRF nº 14, de 1º de dezembro de 2005.txt',
       'data/normas_referencia

In [82]:
normas_json = {}

for which_file in all_files:
    with open(which_file) as input_file:
        normas_json[os.path.basename(which_file)] = input_file.read()

In [83]:
normas_json.keys()

dict_keys(['ADI RFB nº 12, de 2016.txt', 'Acordo para Evitar a Dupla Tributação em Matéria de Impostos sobre a Renda e o Capital firmado entre o Brasil e a Alemanha.txt', 'Acórdão do RE nº 855.091RS (Tema 808).txt', 'Ato Declaratório (AD) PGFN nº 3, de 18 de setembro de 2008.txt', 'Ato Declaratório Cosar nº 47.txt', 'Ato Declaratório Executivo Codac nº 23, de 4 de dezembro de 2019.txt', 'Ato Declaratório Executivo RFB nº 2, de 6 de março de 2024.txt', 'Ato Declaratório Executivo SRF nº 72, de 22 de dezembro de 2005.txt', 'Ato Declaratório Interpretativo RFB nº 1.txt', 'Ato Declaratório Interpretativo SRF nº 14, de 1º de dezembro de 2005.txt', 'Ato Declaratório Interpretativo SRF nº 14, de 4 de maio de 2004.txt', 'Ato Declaratório Interpretativo SRF nº 16, de 22 de dezembro de 2005.txt', 'Ato Declaratório Interpretativo SRF nº 2.txt', 'Ato Declaratório Interpretativo SRF nº 24.txt', 'Ato Declaratório Interpretativo SRF nº 5.txt', 'Ato Declaratório Normativo CST nº 10, de 1991.txt', 'Ato

In [107]:
hierarchical_text_splitter = HierachicalRecursiveCharacterTextSplitter(
    separators=LEGISLATION_SPLITTING_HIERARCHY_BRAZIL_NO_NEWLINE,
    is_separator_regex=True,
    chunk_size=1000,  # Tamanho máximo do chunk em caracteres
    chunk_overlap=100,  # Sobreposição entre chunks para contexto
    apply_chunk_size=6 # Separator index to start applying the chunk size check.
)

In [108]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # Tamanho máximo do chunk em caracteres
    chunk_overlap=100,  # Sobreposição entre chunks para contexto
)

In [109]:
tokens_counter = tokensCounter("stjiris/bert-large-portuguese-cased-legal-tsdae")

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


### Process (segmentation and flattening) all the referenced documents

In [110]:
from tqdm import tqdm

In [111]:
applied_splitter = []
passages_referred_docs = []

for filename, doc in tqdm(normas_json.items(), desc="Processing all referred legal documents"):
    # print(f"Processing file: {filename}")
    
    # Remove a extensão ".txt" do nome do arquivo
    doc_name = filename.replace(".txt", "")

    # First try applying the hierarchical splitter
    chunks = hierarchical_text_splitter.split_text(doc)
    passages = []
    current_path = ""

    flatten_passages_hierarchy(chunks, current_path, passages)
    
    # print(chunks.keys())
    
    total_statistics = tokens_counter.passages_statistics([passage['passage'] for passage in passages])

    # print(total_statistics)

    #
    # Check if there are indications the hierarchical splitter did not work
    # verifying if there is any chunk longer than the maximum size.
    #
    
    if total_statistics['max_tokens'] > 512:
        # For such document, apply a regular text splitter.
        
        print(f">>> Applying regular text splitter in file {filename}.")

        chunks = text_splitter.split_text(doc)
        applied_splitter.append({'filename': filename,
                                 'splitter': 'text'})

        for passage in chunks:
            # Formata cada chunk com o nome do arquivo e o conteúdo
            passage_with_path = "{}: {}".format(doc_name, passage)
            passages_referred_docs.append(passage_with_path)    
    else:
        applied_splitter.append({'filename': filename,
                                 'splitter': 'hierarchical'})
    
        for passage in passages:
            # Formata cada chunk com o nome do arquivo e o conteúdo
            passage_with_path = "{}: {}".format(doc_name + "_" + passage['path'], passage['passage'])
            passages_referred_docs.append(passage_with_path)

Processing all referred legal documents:   0%|                                                                                                                                            | 0/478 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/scipy/stats/_stats_py.py:1087: RuntimeWarning: divide by zero encountered in divide
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero
/opt/conda/lib/python3.10/site-packages/scipy/stats/_stats_py.py:1087: RuntimeWarning: invalid value encountered in scalar multiply
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero
Processing all referred legal documents:  10%|█████████████▌                                                                                                                    | 50/478 [00:00<00:00, 495.47it/s]

>>> Applying regular text splitter in file Acordo para Evitar a Dupla Tributação em Matéria de Impostos sobre a Renda e o Capital firmado entre o Brasil e a Alemanha.txt.
>>> Applying regular text splitter in file Acórdão do RE nº 855.091RS (Tema 808).txt.
>>> Applying regular text splitter in file Ato Declaratório Normativo Cosit nº 24, de 14 de setembro de 1999.txt.
>>> Applying regular text splitter in file Ato Declaratório PGFN Nº 1, de 12 de março de 2018.txt.
>>> Applying regular text splitter in file Ato Declaratório PGFN nº 5, de 3 de maio de 2016.txt.
>>> Applying regular text splitter in file Ato Declaratório RFB nº 3.txt.
>>> Applying regular text splitter in file Ato Declaratório do Presidente da Mesa do Congresso Nacional nº 38, de 14 de outubro de 2005.txt.
>>> Applying regular text splitter in file Ação Direta de Inconstitucionalidade nº 5.583DF do Supremo Tribunal Federal (STF).txt.
>>> Applying regular text splitter in file Consolidação das Leis do Trabalho (CLT).txt.


Processing all referred legal documents:  21%|██████████████████████████▉                                                                                                      | 100/478 [00:00<00:03, 114.69it/s]

>>> Applying regular text splitter in file Decreto nº 75.102.txt.
>>> Applying regular text splitter in file Decreto nº 8.624.txt.
>>> Applying regular text splitter in file Decreto nº 85.306.txt.
>>> Applying regular text splitter in file Decreto nº 86.084.txt.
>>> Applying regular text splitter in file Decreto nº 86.365.txt.
>>> Applying regular text splitter in file Decreto nº 87.563.txt.
>>> Applying regular text splitter in file Documento de Informação e Apuração do ITR (Diat).txt.
>>> Applying regular text splitter in file IN RFB nº 1.500, de 2014.txt.
>>> Applying regular text splitter in file Instrução Normativa RFB nº 1.037.txt.
>>> Applying regular text splitter in file Instrução Normativa RFB nº 1.500.txt.


Processing all referred legal documents:  26%|█████████████████████████████████▉                                                                                                | 125/478 [00:01<00:05, 61.97it/s]

...

>>> Applying regular text splitter in file Parecer Normativo CST nº 32, de 17 de agosto de 1981.txt.
>>> Applying regular text splitter in file Parecer Normativo CST nº 36, de 30 de maio de 1977.txt.
>>> Applying regular text splitter in file Parecer Normativo CST nº 6, de 19 de fevereiro de 1986.txt.
>>> Applying regular text splitter in file Parecer Normativo CST nº 60, de 20 de junho de 1978.txt.
>>> Applying regular text splitter in file Parecer Normativo CST nº 62, de 31 de agosto de 1976.txt.
>>> Applying regular text splitter in file Parecer Normativo CST nº 66, de 5 de setembro de 1986.txt.
>>> Applying regular text splitter in file Parecer Normativo CST nº 68, de 14 de setembro de 1976.txt.
>>> Applying regular text splitter in file Parecer Normativo CST nº 72, de 1979.txt.
>>> Applying regular text splitter in file Parecer Normativo CST nº 90, de 16 de outubro de 1978.txt.
>>> Applying regular text splitter in file Parecer Normativo Cosit nº 1, de 24 de setembro de 2002.txt.


Processing all referred legal documents:  72%|██████████████████████████████████████████████████████████████████████████████████████████████                                    | 346/478 [00:05<00:01, 93.86it/s]

>>> Applying regular text splitter in file Parecer nº 93 2018 DECOR CGU AGU.txt.
>>> Applying regular text splitter in file Portaria Conjunta SRFTSE nº 74, de 10 de janeiro de 2006.txt.
>>> Applying regular text splitter in file Portaria nº 277.txt.
>>> Applying regular text splitter in file REsp nº 1.306.393DF_ Tema Repetitivo nº 535.txt.
>>> Applying regular text splitter in file Resolução CGPC nº 26, de 29 de setembro de 2008.txt.
>>> Applying regular text splitter in file Resolução CGSN nº 140, de 22 de maio de 2018.txt.
>>> Applying regular text splitter in file Resolução TSE nº 22.250, de 2006.txt.
>>> Applying regular text splitter in file Sistema de Recolhimento Mensal Obrigatório (Carnê-Leão).txt.
>>> Applying regular text splitter in file Solução de Consulta Cosit nº 105.txt.
>>> Applying regular text splitter in file Solução de Consulta Cosit nº 115.txt.
>>> Applying regular text splitter in file Solução de Consulta Cosit nº 135.txt.


Processing all referred legal documents: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 478/478 [00:05<00:00, 80.03it/s]

>>> Applying regular text splitter in file Solução de Consulta Cosit nº 144.txt.
>>> Applying regular text splitter in file Solução de Consulta Cosit nº 15.txt.
>>> Applying regular text splitter in file Solução de Consulta Cosit nº 173.txt.
>>> Applying regular text splitter in file Solução de Consulta Cosit nº 181.txt.
>>> Applying regular text splitter in file Solução de Consulta Cosit nº 206.txt.
>>> Applying regular text splitter in file Solução de Consulta Cosit nº 256.txt.
>>> Applying regular text splitter in file Solução de Consulta Cosit nº 264, de 24 de junho de 2019.txt.
>>> Applying regular text splitter in file Solução de Consulta Cosit nº 264, de 25 de setembro de 2019.txt.
>>> Applying regular text splitter in file Solução de Consulta Cosit nº 48, de 24 de março de 2021.txt.
>>> Applying regular text splitter in file Solução de Consulta Cosit nº 60, de 23 de junho de 2020.txt.
>>> Applying regular text splitter in file Solução de Consulta Cosit nº 63.txt.
>>> Applying r

In [112]:
splitter_df = pd.DataFrame(applied_splitter)

In [113]:
splitter_df.groupby("splitter").count()

,filename
splitter,
hierarchical,319
text,159


In [114]:
len(passages_referred_docs)

27900

In [115]:
len(passages_referred_docs)

27900

#### Check data statistics

In [116]:
tokens_counter.passages_statistics(passages_referred_docs)

{'max_tokens': 971,
 'min_tokens': 15,
 'mean_tokens': 157.3652688172043,
 'std_tokens': 108.53451531127307,
 'skewness_tokens': 1.2022322856693686,
 'kurtosis_tokens': 2.812744568385634,
 'median_tokens': 121.0,
 'max_token_passage': 18865}

In [117]:
import pickle

In [118]:
with open(DATA_PATH + "chunks_complete_normas_new_20250310.pkl", "wb") as output_file:
    pickle.dump({"passages": passages_referred_docs,
                 "splitter": applied_splitter}, output_file, pickle.HIGHEST_PROTOCOL)